In [6]:
import requests
import json

api_key = 'sBfMQZWan1rpr3ugchVmpKLV_mIw9-GUMnNF52cznlGZc7GINf2J3TBP0We9j8fyIPFMsZP_if7E6H0k72AvPsMU8wwL-hOXWbW2zuRkSEa4TmBEb171AGvtbOecXXYx'
headers = {'Authorization': 'Bearer %s' % api_key}

url = 'https://api.yelp.com/v3/businesses/search'
params = {'term':'bookstore','location':'Brooklyn'}

req = requests.get(url, params=params, headers=headers)

parsed = json.loads(req.text)

businesses = parsed["businesses"]

for business in businesses:
    print("Name:", business["name"])
    print("Rating:", business["rating"])
    print("Price:",business["price"])
    print("Address:", " ".join(business["location"]["display_address"]))
    print("Phone:", business["phone"])
    print("\n")



Name: Unnameable Books
Rating: 4.0
Price: $$
Address: 600 Vanderbilt Ave Brooklyn, NY 11238
Phone: +17187891534


Name: Greenlight Bookstore
Rating: 4.5
Price: $$
Address: 686 Fulton St Brooklyn, NY 11217
Phone: +17182460200


Name: Community Bookstore
Rating: 4.0
Price: $$
Address: 143 7th Ave Brooklyn, NY 11215
Phone: +17187833075


Name: WORD
Rating: 4.5
Price: $$
Address: 126 Franklin St Brooklyn, NY 11222
Phone: +17183830096


Name: Greenlight Bookstore
Rating: 4.5
Price: $$
Address: 632 Flatbush Ave Parkline Tower Brooklyn, NY 11225
Phone: +17182460200


Name: Better Read Than Dead
Rating: 5.0
Price: $
Address: 867 Broadway Bushwick, NY 11206
Phone: +13474705367


Name: Human Relations
Rating: 5.0
Price: $
Address: 1067 Flushing Ave Brooklyn, NY 11237
Phone: 


Name: Molasses Books
Rating: 4.5
Price: $
Address: 770 Hart St Brooklyn, NY 11237
Phone: 


Name: Terrace Books
Rating: 4.0
Price: $$
Address: 242 Prospect Park W Brooklyn, NY 11215
Phone: +17187883475


Name: Spoonbill & 